In [1]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path

# Try to locate repo root that contains `src/`
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features
# --- NEUER MODELL-IMPORT ---
from src.models.EN import ForecastModel


In [2]:

import numpy as np, pandas as pd

# --- NEUER MODELLNAME ---
MODEL_NAME = "elastic_net"
outputs_for_model(MODEL_NAME)
print('PROJECT_ROOT =', PROJECT_ROOT)
print(f'Modell {MODEL_NAME} wird getunt.')

# --- Load data ---
y = load_target()  # ΔIP with DatetimeIndex
X = load_ifo_features()  # ifo panel

idx = y.index.intersection(X.index)
y, X = y.loc[idx], X.loc[idx]
print('Shapes:', X.shape, y.shape)



PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell elastic_net wird getunt.
Shapes: (407, 2160) (407,)


In [ ]:
# --- Corr-spec helper ---
def make_corr_spec(kind: str, window: int = 60, lam: float = 0.98) -> dict:
    spec = dict(DEFAULT_CORR_SPEC)
    spec['mode'] = kind
    if kind == 'expanding':
        spec.pop('window', None)
        spec.pop('lambda', None)
    elif kind == 'ewm':
        spec['window'] = int(window)
        spec['lambda'] = float(lam)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")
    return spec

# Für den Testlauf nur eine Corr-Option
corr_options = [
    ("expanding", make_corr_spec("expanding")),
]

# --- Base config (splits & policy as in thesis) ---
def base_cfg() -> GlobalConfig:
    cfg = GlobalConfig()
    cfg.W0_A     = 180
    cfg.BLOCKS_A = [(181,200), (201,220), (221,240)]
    cfg.W0_B     = 240
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg()
print(cfg0)

# --- ANGEPASSTES TUNING-GRID ---

# 1. FE/DR Grid (EN benötigt DR)
fe_dr_grid = []
for dr in [{"dr_method": "pca", "pca_var_target": 0.95, "pca_kmax": 25}]:
    fe = {
        'lag_candidates': (1, 2, 3),
        'top_k_lags_per_feature': 1,
        'use_rm3': False,
        'k1_topk': 30,
        'redundancy_param': 0.90,
    }
    fe.update(dr)
    fe_dr_grid.append(fe)

# 2. Modell-spezifisches Grid (alpha = l1_ratio)
model_grid_params = [
    {"alpha": 0.2},
    #{"alpha": 0.8},
]

# 3. Target Blocks (Wie von Ihnen gewünscht)
target_block_options = [
    None,                         # 1. Nur Ifo-Features (Baseline)
    ["AR1"],                      # 2. Ifo + AR1
    ["Chronos"],                  # 3. Ifo + Chronos
    ["TSFresh"],                  # 4. Ifo + TSFresh
    #["Chronos", "TSFresh"],       # 5. Ifo + Chronos + TSFresh
    ["AR1", "TSFresh", "Chronos"] # 6. Alle zusammen
]

# 4. NEU: Gewichtungs-Optionen (An/Aus)
weighting_options = [
    {"sample_weight_decay": None},   # Option 1: AUS (Expanding, konstantes Gewicht)
    {"sample_weight_decay": 0.98},  # Option 2: AN (Expanding, exponentielles Gewicht)
]
# ---------------------------------

def build_model_grid():
    hp_grid = []
    for corr_tag, corr_spec in corr_options:
        for fe_dr_hp in fe_dr_grid:
            for model_hp in model_grid_params:
                for block_set in target_block_options:
                    # --- NEUE SCHLEIFE für Gewichtung ---
                    for weight_hp in weighting_options:
                        hp = {
                            **fe_dr_hp,
                            **model_hp,
                            **weight_hp, # <-- Neuer HP
                            'corr_tag': corr_tag,
                            'corr_spec': corr_spec,
                            'target_block_set': block_set
                        }
                        hp_grid.append(hp)
    return hp_grid

model_grid = build_model_grid()
print("HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0])
print("Letztes HP-Set:", model_grid[-1])


# --- Stage A/B: EIN Lauf, EIN Ordner ---
shortlist = run_stageA(
    model_name=MODEL_NAME,
    model_ctor=lambda hp: ForecastModel(hp),
    model_grid=model_grid,
    X=X, y=y, cfg=cfg0,
    keep_top_k_final=min(5, len(model_grid)),
    min_survivors_per_block=max(1, int(len(model_grid) / 2)),
)

run_stageB(
    model_name=MODEL_NAME,
    model_ctor=lambda hp: ForecastModel(hp),
    shortlist=shortlist,
    X=X, y=y, cfg=cfg0,
    # max_months=12,   # optional throttling für schnellen Test
)

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")

GlobalConfig(seed=123, refresh_cadence_months=12, corr_spec={'mode': 'expanding', 'window': None, 'lam': None}, lag_candidates=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), top_k_lags_per_feature=1, use_rm3=True, k1_topk=50, screen_threshold=None, redundancy_method='greedy', redundancy_param=0.9, dr_method='none', pca_var_target=0.95, pca_kmax=25, pls_components=2, W0_A=180, BLOCKS_A=[(181, 200), (201, 220), (221, 240)], W0_B=240, policy_window=24, policy_gain_min=0.03, policy_cooldown=3)
HP-Kombinationen: 10
Erstes HP-Set: {'lag_candidates': (1, 2, 3), 'top_k_lags_per_feature': 1, 'use_rm3': False, 'k1_topk': 30, 'redundancy_param': 0.9, 'dr_method': 'pca', 'pca_var_target': 0.95, 'pca_kmax': 25, 'alpha': 0.2, 'sample_weight_decay': None, 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'lam': None}, 'target_block_set': None}
Letztes HP-Set: {'lag_candidates': (1, 2, 3), 'top_k_lags_per_feature': 1, 'use_rm3': False, 'k1_topk': 30, 'redundancy_param': 0.9, 'dr_method': 'pca', 'pca_var_t